In [ ]:
# Install the package directly from GitHub
!pip install git+https://github.com/wcw100168/Cubed-Sphere-DG-Solver.git

# Advanced Physics: Shallow Water Equations

This tutorial runs a full physics simulation: **Williamson Case 2** (Global Steady State Zonal Flow).
We will use the **NumPy** backend for this demonstration.

In [ ]:
# Use NumPy backend for stability
import os

# Set JAX to CPU if imported indirectly, but we prefer NumPy for this demo
os.environ["JAX_PLATFORMS"] = "cpu"

import numpy as np
import matplotlib.pyplot as plt
from cubed_sphere.solvers import CubedSphereSWE, SWEConfig

In [ ]:
# 1. Stability & Config
N = 16
R = 6.37122e6
H_avg = 10000.0
g = 9.80616

# Calculate max wave speed (Gravity Wave + Flow)
c_wave = np.sqrt(g * H_avg)
u_flow_max = 50.0 # Conservative estimate for Case 2
v_max = c_wave + u_flow_max

# Case 2 Stability
target_cfl = 0.5 # Standard DG CFL
dt = target_cfl * R / (v_max * N**2)

print(f"Wavespeed: {c_wave:.1f} m/s")
print(f"Stable dt: {dt:.3f} s")

config = SWEConfig(
    N=N, 
    R=R, # CRITICAL: Must pass Radius to solver!
    backend='numpy',
    H_avg=H_avg,
    gravity=g,
    dt=dt
)
solver = CubedSphereSWE(config)

In [ ]:
# Setup Williamson Case 2 Initial Condition
# Use the solver's built-in initialization to ensure consistent velocity field
state = solver.get_initial_condition(type="case2")

# Visualize Initial State Statistics
h_min = np.min(state[0])
h_max = np.max(state[0])
print(f"Initial Height range: [{h_min:.4e}, {h_max:.4e}]")
print(f"Initial State Shape: {state.shape}")

> **Note**: This simulation uses Exact (Analytical) Geometry derived from the grid mapping. Minor geometric inconsistencies (~1e-7) in mass conservation are expected due to the Strong Form formulation but the system remains stable for this short demonstration.

In [ ]:
# Run Simulation
t_accumulated = 0.0
n_steps = 20

print("Starting Integration...")
initial_mass = np.sum(state[0])

for step in range(n_steps):
    state = solver.step(t_accumulated, state, dt)
    t_accumulated += dt
    
print("Integration Complete.")

In [ ]:
# Conservation Check
final_mass = np.sum(state[0])
diff = abs(final_mass - initial_mass)
rel_error = diff / initial_mass

print(f"Initial Mass: {initial_mass:.5e}")
print(f"Final Mass:   {final_mass:.5e}")
print(f"Mass Error:   {diff:.5e}")
print(f"Rel Error:    {rel_error:.5e}")

# Check relative error (should be near machine precision for mass)
assert rel_error < 1e-10, "Mass should be conserved!"

In [ ]:
# Plot Height Field
# Access geometry from the solver implementation
faces = solver._impl.faces
# Get the name of the first face (Index 0)
face_name = solver._impl.topology.FACE_MAP[0] # Usually 'P1'

# Unwrap Face 0 (P1)
# State shape: (Vars, Faces, Xi, Eta) -> (0, 0)
h_field = state[0, 0] / faces[face_name].sqrt_g 

plt.figure(figsize=(8, 6))
plt.imshow(h_field, extent=[-1, 1, -1, 1], origin='lower', cmap='viridis')
plt.colorbar(label='Geopotential Height (m)')
plt.title(f"Block 0 ({face_name}) Height Field at t={t_accumulated:.1f}s")
plt.show()